# CORDEX domains

The domain module should give some tools to work with preconfigured or user defined domains. Domains are defined as xarray datasets that will contain dimensions and coodinates according to CF-conventions.

**NOTE**: The domain module mostly focuses on working with rotated cordex domains and how they are defined in the [cordex archive specifications](https://is-enes-data.github.io/cordex_archive_specifications.pdf). However, there are some regional models that use different mappings instead of `rotated_pole` or `rotated_latitude_longitude` which we focus on. Any expertise working with those different mappings is highly welcome!

In [ ]:
import cordex as cx

The domain module contains some useful functions to work with cordex meta data, e.g., you can get some domain grid information using

In [ ]:
cx.domain_info("EUR-11")

All available cordex domains can be found in the [CORDEX domain table on github](https://github.com/WCRP-CORDEX/domain-tables) or directly from withing py-cordex, e.g.

In [ ]:
cx.domains.table

## `EUR-11` example

The heart of the module are some functions that create a dataset from the grid information, e.g.

In [ ]:
eur11 = cx.cordex_domain("EUR-11", dummy="topo")
eur11

The `dummy='topo'` argument means, we want a dummy variable in the dataset to see how the domain looks like. For the dummy topography, we use the `cdo topo` operator in the background. So maybe you have to install `python-cdo`, e.g., `conda install -c conda-forge python-cdo`. Working with xarray datasets means, that we can use all the nice functions of xarray including plotting, e.g.,

In [ ]:
eur11.topo.plot(cmap="terrain")

py-cordex also provides some [xarray accessor functions](https://docs.xarray.dev/en/stable/internals/extending-xarray.html). E.g., to get an overview of a domain, you can use the `ds.cx.map` function:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
eur11.cx.map()

Let's define a slightly more sophisticated plotting function that uses cartopy for the right [projection](https://scitools.org.uk/cartopy/docs/latest/tutorials/understanding_transform.html) with a rotated pole:

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt


def plot(
    da,
    transform=ccrs.PlateCarree(),
    projection=ccrs.PlateCarree(),
    vmin=None,
    vmax=None,
    borders=True,
    xlocs=range(-180, 180, 10),
    ylocs=range(-90, 90, 5),
    extent=None,
    figsize=(15, 10),
    title="",
):
    """plot a domain using the right projections and transformations with cartopy"""
    plt.figure(figsize=figsize)
    ax = plt.axes(projection=projection)
    if extent:
        ax.set_extent(extent, crs=projection)
    ax.gridlines(
        draw_labels=True, linewidth=0.5, color="gray", xlocs=xlocs, ylocs=ylocs
    )
    da.plot(ax=ax, cmap="terrain", transform=transform, vmin=vmin, vmax=vmax)
    ax.coastlines(resolution="50m", color="black", linewidth=1)
    if borders:
        ax.add_feature(cf.BORDERS)
    return ax

We can now use the grid mapping information to plot the data in it's native coordinate system using cartopy:

In [ ]:
pole = (
    eur11.rotated_latitude_longitude.grid_north_pole_longitude,
    eur11.rotated_latitude_longitude.grid_north_pole_latitude,
)
pole

In [ ]:
plot(eur11.topo, transform=ccrs.RotatedPole(*pole), projection=ccrs.RotatedPole(*pole))

## User defined domain

The domains are actually created from a csv table that define standard cordex domains. E.g., the data used to created the `EUR-11` domain is

In [ ]:
cx.domains.table.loc["EUR-11"]

The domains are created using the [create_dataset](https://py-cordex.readthedocs.io/en/latest/generated/cordex.create_dataset.html) function, e.g.:

In [ ]:
eur11_user = cx.create_dataset(
    nlon=424,
    nlat=412,
    dlon=0.11,
    dlat=0.11,
    ll_lon=-28.375,
    ll_lat=-23.375,
    pollon=-162.00,
    pollat=39.25,
    dummy="topo",
)

We can check that this gives the same result as our preconfigured domain.

In [ ]:
eur11_user.equals(eur11)